In [3]:
#Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.ticker import PercentFormatter
import seaborn as sns
import datetime as dt
import time
from datetime import date, timedelta
from sklearn.preprocessing import StandardScaler
from tqdm import tqdm
from scipy.stats import poisson
import scipy.stats as stats
%matplotlib inline

from google.oauth2.service_account import Credentials
import google_auth_httplib2
import pygsheets
import df2gspread as d2g
import googleapiclient
from googleapiclient  import discovery
from httplib2 import Http
from oauth2client import file, client, tools
from gspread_formatting.dataframe import format_with_dataframe, BasicFormatter
from gspread_formatting import Color
import logging

from oauth2client.service_account import ServiceAccountCredentials
from google.oauth2 import service_account
import gslides
from gslides import (Frame,Presentation,Spreadsheet,Table,Series,Chart)

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)
import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.filterwarnings("ignore")

import sqlalchemy as db
from sqlalchemy import create_engine
import mysql.connector
import psycopg2

In [4]:
# GBQ logging
import logging
logger = logging.getLogger('pandas_gbq')
logger.setLevel(logging.DEBUG)
logger.addHandler(logging.StreamHandler())

In [8]:
# GBQ service account connection
credentials=Credentials.from_service_account_file('C:/Users/mncedisi.mncwabe/Documents/Connect to BQ/tal-production-data-4390ad039b75.json')

In [6]:
# GBQ service account connection
cre_marketing=Credentials.from_service_account_file('C:/Users/mncedisi.mncwabe/Downloads/tal-production-marketing-e93d689c8a94.json')

In [9]:
start_time = time.time()

query_no_lvl1 = '''

SELECT customer_id, FY, Auth_Date,Division, Department, Level1, Customer_seq,
DATE_DIFF(Auth_Date,Lag,DAY) AS Diff, RepeatPurchase
FROM (
SELECT customer_id, FY, Auth_Date,Division, Department, Level1, Customer_seq,
 LAG(Auth_Date) OVER (PARTITION BY customer_id ORDER BY  Auth_Date) AS Lag,
CASE WHEN customer_seq > 1 THEN 'Repeat Customer'
ELSE 'New Customer'
END AS RepeatPurchase
FROM (
    SELECT  *,
         CASE WHEN DATE(x.Auth_Date) >= '2019-04-01' AND DATE(x.Auth_Date) < '2020-04-01' THEN "FY20"
              WHEN DATE(x.Auth_Date) >= '2020-04-01' AND DATE(x.Auth_Date) < '2021-04-01' THEN "FY21"
              WHEN DATE(x.Auth_Date) >= '2021-04-01' AND DATE(x.Auth_Date) < '2022-04-01' THEN "FY22"
      END AS FY,
    RANK() OVER (PARTITION BY customer_id ORDER BY x.Auth_Date) AS customer_seq
    FROM
(
SELECT  
       DISTINCT customer_id,
        bh.bh_division AS Division, 
        bh.bh_reporting_department AS Department,
        bh.bh_level_1 AS Level1,
       DATE(o.order_authed_timestamp) AS Auth_Date,
        o.order_id,
        oi.order_item_sales AS SalesEx,
        oi.order_item_quantity AS Units
    FROM `tal-production-data-bi.tal_dm_order.orders_2010_2020` o,
        UNNEST(o.order_items) oi
    INNER JOIN `tal-production-data-bi.tal_dm_product.business_hierarchy` bh
        ON bh.product_id = oi.product_id
    WHERE   
        DATE(o.order_placed_timestamp) >= "2019-04-01" AND DATE(o.order_placed_timestamp) < "2020-01-01" 
        AND DATE(o.order_authed_timestamp) >= "2019-04-01" AND DATE(o.order_authed_timestamp) < "2020-01-01" 
        AND o.order_authorisation_status IN ('Auth')
        AND o.is_not_return_cancel_in_full IS TRUE
        #AND o.voucher_sales_percent =0
        AND is_test_customer = false
        AND o.order_authed_timestamp IS NOT NULL

UNION ALL

SELECT 
        DISTINCT customer_id,
        bh.bh_division AS Division, 
        bh.bh_reporting_department AS Department,
        bh.bh_level_1 AS Level1,
        DATE(o.order_authed_timestamp) AS Auth_Date,
        o.order_id,
        oi.order_item_sales AS SalesEx,
        oi.order_item_quantity AS Units
    FROM `tal-production-data-bi.tal_dm_order.orders_2020_2030` o,
        UNNEST(o.order_items) oi
    INNER JOIN `tal-production-data-bi.tal_dm_product.business_hierarchy` bh
        ON bh.product_id = oi.product_id
    WHERE   
        DATE(o.order_placed_timestamp) >= "2020-01-01" AND DATE(o.order_placed_timestamp) < "2022-06-01" 
        AND DATE(o.order_authed_timestamp) >= "2020-01-01" AND DATE(o.order_authed_timestamp) < "2022-06-01" 
        AND o.order_authorisation_status IN ('Auth')
        AND o.is_not_return_cancel_in_full IS TRUE
        #AND o.voucher_sales_percent =0
        AND is_test_customer = false
        AND o.order_authed_timestamp IS NOT NULL) x)
WHERE FY IS NOT NULL 
  #AND customer_id IN (8)
GROUP BY 1,2,3,4,5,6,7)
'''
df_no_lvl1 = pd.read_gbq(query_no_lvl1, project_id='gcp-takealot', dialect='standard', credentials=credentials,progress_bar_type='tqdm_notebook')

print("--- %s seconds ---" % (time.time() - start_time))

Requesting query... 
Requesting query... 
2022-06-17 12:18:03,466 : DEBUG : Requesting query... 
Query running...
Query running...
2022-06-17 12:18:07,762 : DEBUG : Query running...
Job ID: 12bb69e4-020c-4044-94fd-db1633af2fbe
Job ID: 12bb69e4-020c-4044-94fd-db1633af2fbe
2022-06-17 12:18:07,776 : DEBUG : Job ID: 12bb69e4-020c-4044-94fd-db1633af2fbe
  Elapsed 14.63 s. Waiting...
  Elapsed 14.63 s. Waiting...
2022-06-17 12:18:18,097 : INFO :   Elapsed 14.63 s. Waiting...
  Elapsed 25.22 s. Waiting...
  Elapsed 25.22 s. Waiting...
2022-06-17 12:18:28,689 : INFO :   Elapsed 25.22 s. Waiting...
  Elapsed 35.33 s. Waiting...
  Elapsed 35.33 s. Waiting...
2022-06-17 12:18:38,792 : INFO :   Elapsed 35.33 s. Waiting...
  Elapsed 45.45 s. Waiting...
  Elapsed 45.45 s. Waiting...
2022-06-17 12:18:48,919 : INFO :   Elapsed 45.45 s. Waiting...
Query done.
Processed: 7.2 GB Billed: 7.2 GB
Query done.
Processed: 7.2 GB Billed: 7.2 GB
2022-06-17 12:18:57,353 : DEBUG : Query done.
Processed: 7.2 GB Bil

Downloading:   0%|          | 0/57413470 [00:00<?, ?rows/s]

Got 57413470 rows.

Got 57413470 rows.

2022-06-17 14:28:17,428 : DEBUG : Got 57413470 rows.

Total time taken 7813.96 s.
Finished at 2022-06-17 14:28:17.
Total time taken 7813.96 s.
Finished at 2022-06-17 14:28:17.
2022-06-17 14:28:17,428 : INFO : Total time taken 7813.96 s.
Finished at 2022-06-17 14:28:17.


--- 7814.431211948395 seconds ---


In [10]:
#df_no_lvl1_binned
start_time = time.time()

df_no_lvl1.to_gbq(destination_table='tal-production-marketing.rfmd_data.cust_journey_raw',
               project_id='tal-bq-job-default',
               credentials=cre_marketing,
               chunksize=100000,
               progress_bar=True,
               if_exists='replace')

print("--- %s seconds ---" % (time.time() - start_time))

57413470 out of 57413470 rows loaded.
57413470 out of 57413470 rows loaded.
57413470 out of 57413470 rows loaded.
100%|██████████| 1/1 [00:00<?, ?it/s]

--- 964.6557521820068 seconds ---
